In [15]:
#웹 크롤링 라이브러리(beautifulsoup4) 설치 
!pip install beautifulsoup4
#selenium 설치
!pip install selenium
#webdriver-manager 설치
!pip install webdriver-manager

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
import os
import sys
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [2]:
# 데이터 저장 폴더 생성
import os
if not os.path.exists("crawled_data"):
    os.makedirs("crawled_data")

In [3]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# Service 객체를 사용하여 드라이버 초기화
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(3)

In [4]:
import json

route = "./api_key.json"

# JSON 파일에서 API 키 읽기
with open(route, "r", encoding="utf-8") as file:
    api_keys = json.load(file)
try:
    client_id = api_keys["client_id"]
    client_secret = api_keys["client_secret"]
except KeyError:
    print("API 키가 JSON 파일에 없습니다. 확인 후 다시 시도하세요.")
    sys.exit(1)
else:
    print("API 키를 성공적으로 읽었습니다.")


API 키를 성공적으로 읽었습니다.


In [5]:
# selenium으로 검색 페이지 불러오기 #
naver_urls = []
postdate = []
titles = []

In [ ]:
# 검색어 입력 (키워드가 바뀌는 경우 한번 더 실행해 주세요)
keword = input("검색할 키워드를 입력해주세요:")
encText = urllib.parse.quote(keword)

In [7]:
# 검색을 끝낼 페이지 입력
end = input("\n크롤링을 끝낼 위치를 입력해주세요. (기본값:1, 최대값:1000):")  
if end == "":
    end = 1
else:
    end = int(end)
print("\n 1 ~ ", end, "페이지 까지 크롤링을 진행 합니다")

# 1로 설정하지 않으면 중복 데이터 발생 가능성 있음


 1 ~  1 페이지 까지 크롤링을 진행 합니다


In [8]:
# 한번에 가져올 페이지 입력
display = input("\n한번에 가져올 페이지 개수를 입력해주세요.(기본값:10, 최대값: 100):")
if display == "":
    display = 10
else:
    display = int(display)
print("\n한번에 가져올 페이지 : ", display, "페이지")


한번에 가져올 페이지 :  100 페이지


In [ ]:
# 크롤링 시작 (키워드와 페이지 수 설정 후 실행 해주세요)

for start in range(end):
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + "&start=" + str(start+1) + "&display=" + str(display) # JSON 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        
        data = json.loads(response_body.decode('utf-8'))['items']
        for row in data:
            if('blog.naver' in row['link']):
                naver_urls.append(row['link'])
                postdate.append(row['postdate'])
                title = row['title']
                # html태그제거
                pattern1 = '<[^>]*>'
                title = re.sub(pattern=pattern1, repl='', string=title)
                titles.append(title)
        time.sleep(2)
    else:
        print("Error Code:" + rescode)

In [10]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

contents = []  # 블로그 내용을 저장할 리스트

try:
    for i, url in enumerate(naver_urls):
        print(f"블로그 URL ({i + 1}/{len(naver_urls)}): {url}")
        driver.get(url)
        time.sleep(3)  # 페이지 로딩 대기

        try:
            # iframe으로 전환
            iframe = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "mainFrame"))
            )
            driver.switch_to.frame(iframe)

            # iframe 내부의 HTML 가져오기
            source = driver.page_source
            html = BeautifulSoup(source, "html.parser")

            # 블로그 내용 추출
            content = html.select_one("div.se-main-container")  # 스마트에디터 블로그
            if content:
                content = content.get_text(strip=True)
            else:
                # 일반 블로그 처리
                content = html.select_one("div#postViewArea")
                content = content.get_text(strip=True) if content else "내용을 가져올 수 없습니다."

            contents.append(content)

        except Exception as e:
            print(f"오류 발생: {e}")
            contents.append("error")  # 오류 발생 시 'error' 추가

        finally:
            # iframe에서 기본 컨텍스트로 전환
            driver.switch_to.default_content()

    # 리스트 길이 동기화 확인
    while len(contents) < len(titles):
        contents.append("error")  # 누락된 데이터를 'error'로 채움

    # 데이터프레임 생성
    news_df = pd.DataFrame({'title': titles, 'content': contents, 'date': postdate})
    news_df.to_csv(/f'./crawled_data/{keword}_blog.csv', index=False, encoding='utf-8-sig')

except Exception as e:
    print(f"전체 오류 발생: {e}")

블로그 URL (1/100): https://blog.naver.com/tlsskwksk123/223887541748
블로그 URL (2/100): https://blog.naver.com/livelyej/223882290256
블로그 URL (3/100): https://blog.naver.com/pkhyiu81/223890440726
블로그 URL (4/100): https://blog.naver.com/vkdnjtmffka/223742661559
블로그 URL (5/100): https://blog.naver.com/milkmilk33/223800615031
블로그 URL (6/100): https://blog.naver.com/daaoll_/223885251384
블로그 URL (7/100): https://blog.naver.com/goldsun17_/223865784587
블로그 URL (8/100): https://blog.naver.com/eeeeeeeer00/223650848050
블로그 URL (9/100): https://blog.naver.com/blueblue2016/223886068020
블로그 URL (10/100): https://blog.naver.com/shmwlove6/223890032858
블로그 URL (11/100): https://blog.naver.com/sndorhcl12/223892206927
블로그 URL (12/100): https://blog.naver.com/khe22ll/223637063967
블로그 URL (13/100): https://blog.naver.com/slowdownandenjoylife/223738954369
블로그 URL (14/100): https://blog.naver.com/asd85462/223851644818
블로그 URL (15/100): https://blog.naver.com/hyo_one_01/223824051439
블로그 URL (16/100): https://blog.

In [11]:
news_df #18500

,title,content,date
0,충북 증평 맛집 [ 브라운면 증평본점 ] 줄 서서 먹는 칼국수와...,​브라운면 증평본점​충북 증평 맛집​​​안녕하세요!​지인분께서 SNS 보시다가갑자기...,20250605
1,증평맛집 브라운면 방문후기,​초등학교에 입학한 아이의 공개수업이 있는 날이었다.평일에 둘이 함께 쉬는 건 정말...,20250529
2,증평현지인맛집 한우 퀄리티 장난 아니였던곳,요즘 스트레스가 쌓여서 친구랑기분 전환 삼아 드라이브를 다녀왔어요.큰 계획 없이 차...,20250609
3,충북 도너츠 맛집 착한 가성비의 증평 도레아도넛,충북 도너츠 맛집 착한 가성비의 증평 도레아도넛증평 도레아도너츠충북 맛집​충북 증평...,20250130
4,증평 현지인 맛집 춘화루,"#협찬\n\n증평 현지인 맛집 춘화루\n\n탕수육, 짜장면, 짬뽕, 볶음밥 다 성공...",20250318
...,...,...,...
95,"청주) 증평IC 디저트맛집, 다양한 종류의 빵이 있는 [ 하그트 ]","[ 영업 시간 ]월~금 09:00 ~ 21:00토,일 09:00 ~ 22:00마...",20240905
96,청주근교 돈까스 맛집. 내돈내산 돈까스 증평 율리돈까스....,#청주근교나들이#청주근교가볼만한곳#청주근교맛집#청주근교맛집추천#청주근교맛집부모님#청...,20250606
97,증평 가지찜백반맛집 한울마루,마눌님과 주말마다 근처에 대형카페투어 다니는데 지나가다 들른 밥집을 소개해본다.갓...,20240906
98,2024-10-22 증평 보양식 삼계탕 맛집 '금송가든 한방백숙...,증평 보양식 삼계탕맛집'금송가든 한방백숙'아기랑 가기 좋은식당스튜디오은은 본점​증평...,20241022


In [ ]:

news_df['date'].min()

'20240219'